In [1]:
import pandas as pd 
import logging

from dotenv import load_dotenv
import openai
import pandas as pd

from strategies import StrategyRating
from utils import StatsHolder

load_dotenv('../openai.env')
openai.api_key  =None# os.getenv('API_KEY')
logger = logging.getLogger()

In [5]:
stats = StatsHolder.from_csv(path_docs='resource/boxscoretraditionalv2/')
stats.add_strategy(StrategyRating)
s = StrategyRating
s.top =1
stats.add_strategy(s)
new_record = pd.read_csv('boxscoretraditionalv2_0_0012100003.csv') 
stats.add_record(new_record)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:02<00:00, 40.24it/s]
2023-10-31 16:57:50,059 - INFO - Added "StrategyRating Top-10" strategy
2023-10-31 16:57:50,537 - INFO - Added "StrategyRating Top-10" strategy
2023-10-31 16:57:50,551 - INFO - added game_id: 12100003.0
2023-10-31 16:57:51,086 - INFO - {'description': 'Top 5 players', 'values': {'in_top': {'FGM': {201939}, 'FGA': {1628369}, 'DREB': {1630596}, 'BLK': {1629057}}, 'out_top': {'FGM': {1629640}, 'FGA': {203897}, 'DREB': {203944}, 'BLK': {201572}}}}


In [7]:
strategies =  ' '.join(stats.show_strategies())
print(f'all strategy: {strategies}')
stats.strategies['StrategyRating Top-10'].show_data_strategy(10)

all strategy: StrategyRating Top-10


{'description': 'Top 10 players',
 'values': {'in_top': {'FGA': {1628369},
   'FG3M': {203897, 1627741},
   'FT_PCT': {1627846},
   'DREB': {1628976},
   'REB': {1628976},
   'STL': {204456}},
  'out_top': {'FGA': {1630163},
   'FG3M': {203468, 1629670},
   'FT_PCT': {1629639},
   'DREB': {203999},
   'REB': {1628381},
   'STL': {1630163}}}}

In [31]:
prompt = ''
  
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]
  
a = get_completion(prompt, model="gpt-3.5-turbo")
print(a)

Структура диалога:

I. Подготовка к игре
   A. Ознакомление с правилами
      1. Вопрос о знании правил
         - Игорь Охотников: "Ознакомлены ли вы с содержанием статей 'Об Игре' и 'О Клубе' и особенно с этическими правилами в статье 'Об Игре'?"
         - Яся Маратова: "1) да 2) не очень, сейчас все прочитаю"
      2. Подтверждение согласия с правилами
         - Игорь Охотников: "Согласны ли вы с этими правилами, готовы ли неукоснительно соблюдать их в ходе Игры? Есть ли какие-то сомнения или непонятные моменты в них?"
         - Яся Маратова: "Да, конечно согласна."
   B. Подтверждение слова чести
      - Игорь Охотников: "1) Вы согласны, что человек, подписавшийся под этими правилами, а затем злостно их нарушающий - человек непорядочный? 2) Даёте ли вы слово чести, что не будете так делать?"
      - Яся Маратова: "1. Да, конечно. 2. Да."
   C. Подтверждение понимания статьи о чести
      - Игорь Охотников: "Прочитайте еще раз статью о чести. Если вы согласны с ней, то повторите 